In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import math
import trieste
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from mo_lp_true_pf.generate_true_pareto_fronts import read_true_pf
from mo_lp.experiment import run_experiment, get_hv_regret

## Simple 1d input case

In [ ]:
search_space = trieste.space.Box([0], [2*math.pi])

def f1(x):
    return tf.cos(2 * x) + tf.sin(x)

def f2(x):
    return 0.2 * (tf.cos(x) - tf.sin(x)) + 0.3

def f(x):
    return tf.concat([f1(x), f2(x)], axis=-1)

In [ ]:
num_initial_points = 3
num_query_points = 4
num_steps = 3
n_obj = 2

n_experiments = 10

In [ ]:
from mo_lp_true_pf.generate_true_pareto_fronts import SIMPLE_1D_INPUT_FILENAME
true_pf = read_true_pf(os.path.join("mo_lp_true_pf", SIMPLE_1D_INPUT_FILENAME))

batch_mc_hv_regret = []
mo_lp_hv_regret = []
for _ in range(n_experiments):
    batch_mc_observations, mo_lp_observations = run_experiment(f,
                                                                n_obj,
                                                                search_space,
                                                                num_initial_points,
                                                                num_steps,
                                                                num_query_points)

    batch_mc_hv_regret.append(get_hv_regret(true_pf, batch_mc_observations, num_initial_points))
    mo_lp_hv_regret.append(get_hv_regret(true_pf, mo_lp_observations, num_initial_points))

In [ ]:
batch_mc_hv_regret = np.array(batch_mc_hv_regret)
mo_lp_hv_regret = np.array(mo_lp_hv_regret)

In [ ]:
x = range(num_steps * num_query_points + 1)

plt.fill_between(x, np.quantile(batch_mc_hv_regret, q=0.25, axis=0), np.quantile(batch_mc_hv_regret, q=0.75, axis=0), facecolor='r', alpha=0.25)
plt.fill_between(x, np.quantile(mo_lp_hv_regret, q=0.25, axis=0), np.quantile(mo_lp_hv_regret, q=0.75, axis=0), facecolor='g', alpha=0.25)
plt.plot(x, np.mean(batch_mc_hv_regret, axis=0), c='r', label='Batch MC')
plt.plot(x, np.mean(mo_lp_hv_regret, axis=0), c='g', label='MO LP')
plt.title(f"Simple 1d input, Hypervolume error, {num_steps} iterations of batch size {num_query_points}")
plt.legend()
plt.show()

## 2d input functions from Gardner et al. 2014

In [ ]:
search_space = trieste.space.Box([0, 0], [2*math.pi, 2*math.pi])

def f1(input_data):
    x, y = input_data[..., -2], input_data[..., -1]
    z = tf.cos(2.0 * x) * tf.cos(y) + tf.sin(x)
    return z[:, None]

def f2(input_data):
    x, y = input_data[:, -2], input_data[:, -1]
    # changes are made so that the function is between 0 and 1
    z = 1.0 - (tf.cos(x) * tf.cos(y) - tf.sin(x) * tf.sin(y) + 1.0) / 2.0
    return z[:, None]

def f(x):
    return tf.concat([f1(x), f2(x)], axis=-1)

In [ ]:
num_initial_points = 3
num_query_points = 4
num_steps = 5
n_obj = 2

n_experiments = 10

In [ ]:
from mo_lp_true_pf.generate_true_pareto_fronts import GARDNER_2D_INPUT_FILENAME
true_pf = read_true_pf(os.path.join("mo_lp_true_pf", GARDNER_2D_INPUT_FILENAME))

batch_mc_hv_regret = []
mo_lp_hv_regret = []
for _ in range(n_experiments):
    batch_mc_observations, mo_lp_observations = run_experiment(f,
                                                                n_obj,
                                                                search_space,
                                                                num_initial_points,
                                                                num_steps,
                                                                num_query_points)

    batch_mc_hv_regret.append(get_hv_regret(true_pf, batch_mc_observations, num_initial_points))
    mo_lp_hv_regret.append(get_hv_regret(true_pf, mo_lp_observations, num_initial_points))

In [ ]:
batch_mc_hv_regret = np.array(batch_mc_hv_regret)
mo_lp_hv_regret = np.array(mo_lp_hv_regret)

In [ ]:
x = range(num_steps * num_query_points + 1)

plt.fill_between(x, np.quantile(batch_mc_hv_regret, q=0.25, axis=0), np.quantile(batch_mc_hv_regret, q=0.75, axis=0), facecolor='r', alpha=0.25)
plt.fill_between(x, np.quantile(mo_lp_hv_regret, q=0.25, axis=0), np.quantile(mo_lp_hv_regret, q=0.75, axis=0), facecolor='g', alpha=0.25)
plt.plot(x, np.mean(batch_mc_hv_regret, axis=0), c='r', label='Batch MC')
plt.plot(x, np.mean(mo_lp_hv_regret, axis=0), c='g', label='MO LP')
plt.title(f"Simple 1d input, Hypervolume error, {num_steps} iterations of batch size {num_query_points}")
plt.legend()
plt.show()